In [8]:
import weaviate
from langchain.document_loaders import DirectoryLoader, WebBaseLoader
import pandas as pd

# Weaviate

In [27]:
# 定义client
client = weaviate.Client(url='http://localhost:8080')
class_name = 'Stephen_Chow'

## weaviate参数定义

In [28]:
class_obj = {
    'class': class_name,         # class的名字
    'vectorIndexConfig':{
        'distance': 'l2-squared',   # 这里的distance是选择向量检索方式，这里选择的是欧式距离
    },
}

## 创建或删除class

In [29]:
# 创建class
client.schema.create_class(class_obj)
# 如果想删除以前的class
# client.schema.delete_class(class_name='area')

# 数据处理

In [16]:
# 导入数据
df = pd.read_csv('data.csv', encoding='GB18030')
df

,sentence
0,人生如果没有梦想，那和咸鱼有什么区别。
1,小强!小强你怎么了小强?小强，你不能死啊!我跟你相依为命，同甘共苦了这么多年，一直把你当亲生...
2,曾经有一份真诚的爱摆在我的面前，但是我没有珍惜，等到失去的时候才后悔莫及，尘世间最痛苦的事莫...
3,屎，你是一滩屎。命比蚁便宜。我开奔驰，你挖鼻屎。吃饭!?吃屎吧你!
4,杀人这种事，我整天都有这种想法的。
5,本来我要用这10元钱念书，当个医生、律师，但为了世界和平……唉!
6,你可以叫我跑龙套的，但是请不要在前面加个“死”字好不好?
7,其实，我是一个演员。
8,要很努力，很努力，才能做到一点点的成功。
9,除暴安良是我们做市民的责任，行善积德是我本身的兴趣，所以扶老太太过马路我每个星期都做一次，如...


In [19]:
# 专成list形式
sentence_data = df.sentence.tolist()

# 定义embeddings模型

In [20]:
# 定义embeddings模型
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('emb_model/text2vec-large-chinese')   # embeddings模型路径

# 句子向量化
sentence_embeddings = model.encode(sentence_data)
# sentence_embeddings = model.encode(sentence_data)
sentence_embeddings

/media/ubuntu_data/LukJH/anaconda/envs/ljh_llm_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name emb_model/text2vec-large-chinese. Creating a new one with MEAN pooling.


array([[ 0.07883023, -0.7958345 , -0.3155929 , ...,  1.2939562 ,
         0.05681886,  0.68244064],
       [-0.46256578,  0.10221019, -0.07999119, ...,  1.5513865 ,
         0.39791292, -0.30931145],
       [-0.5512182 , -0.50856495,  0.6594512 , ..., -0.04717647,
        -0.8090207 , -0.76115984],
       ...,
       [-1.6151336 , -0.07856621,  1.0403824 , ...,  0.508074  ,
         0.26066488, -0.72345275],
       [-0.5566714 , -1.0310702 , -0.78030396, ...,  0.3580936 ,
        -0.9182949 ,  0.28291687],
       [-0.04114265, -0.9631913 ,  0.29949385, ...,  0.48185617,
        -0.47320065,  0.00453359]], dtype=float32)

# 整合数据到DataFrame里

In [30]:
# 将句子和embeddings后的数据整合到DataFrame里面
data = {'sentence':sentence_data,
        'embeddings':sentence_embeddings.tolist()}
df = pd.DataFrame(data)
df

,sentence,embeddings
0,人生如果没有梦想，那和咸鱼有什么区别。,"[0.07883022725582123, -0.795834481716156, -0.3..."
1,小强!小强你怎么了小强?小强，你不能死啊!我跟你相依为命，同甘共苦了这么多年，一直把你当亲生...,"[-0.4625657796859741, 0.10221018642187119, -0...."
2,曾经有一份真诚的爱摆在我的面前，但是我没有珍惜，等到失去的时候才后悔莫及，尘世间最痛苦的事莫...,"[-0.5512182116508484, -0.5085649490356445, 0.6..."
3,屎，你是一滩屎。命比蚁便宜。我开奔驰，你挖鼻屎。吃饭!?吃屎吧你!,"[0.22441639006137848, -0.45846372842788696, -0..."
4,杀人这种事，我整天都有这种想法的。,"[0.025575688108801842, 0.15084435045719147, -1..."
5,本来我要用这10元钱念书，当个医生、律师，但为了世界和平……唉!,"[0.05777275189757347, 0.34411829710006714, -0...."
6,你可以叫我跑龙套的，但是请不要在前面加个“死”字好不好?,"[-1.0173733234405518, -0.7639865875244141, -1...."
7,其实，我是一个演员。,"[0.18424586951732635, -1.0897544622421265, -0...."
8,要很努力，很努力，才能做到一点点的成功。,"[0.23395083844661713, -1.1595344543457031, 1.2..."
9,除暴安良是我们做市民的责任，行善积德是我本身的兴趣，所以扶老太太过马路我每个星期都做一次，如...,"[-0.7226650714874268, 0.02695402503013611, -1...."


# Weaviate数据导入

In [31]:
with client.batch(
    batch_size=100
) as batch:
    for i in range(df.shape[0]):
#         if i%20 == 0:
        print('importing data: {}'.format(i+1))
        # 定义properties
        properties = {
            'sentence_id': i+1,          # 这里是句子id, [1, 2, 3, ...]
            'sentence': df.sentence[i],  # 这里是句子内容
#             'embeddings': df.embeddings[i],
        }
        custom_vector = df.embeddings[i] # 这里是句子向量化后的数据
        # 导入数据
        client.batch.add_data_object(
            properties,
            class_name=class_name,
            vector=custom_vector
        )
print('import completed')

importing data: 1
importing data: 2
importing data: 3
importing data: 4
importing data: 5
importing data: 6
importing data: 7
importing data: 8
importing data: 9
importing data: 10
importing data: 11
importing data: 12
importing data: 13
importing data: 14
importing data: 15
importing data: 16
importing data: 17
importing data: 18
importing data: 19
importing data: 20
import completed


# 用户输入转化

In [32]:
query = model.encode(['除暴安良'])[0].tolist()   # 这里将问题进行embeddings
nearVector = {
    'vector': query
}

In [33]:
response = (
    client.query
    .get(class_name, ['sentence_id', 'sentence']) # 第一个参数为class名字，第二个参数为需要显示的信息
    .with_near_vector(nearVector)             # 使用向量检索，nearVector为输入问题的向量形式
    .with_limit(5)                            # 返回个数(TopK)，这里选择返回5个
    .with_additional(['distance'])            # 选择是否输出距离
    .do()
)

In [34]:
print(json.dumps(response, indent=2))  # 看下输出

{
  "data": {
    "Get": {
      "Stephen_Chow": [
        {
          "_additional": {
            "distance": 769.15155
          },
          "sentence": "\u9664\u66b4\u5b89\u826f\u662f\u6211\u4eec\u505a\u5e02\u6c11\u7684\u8d23\u4efb\uff0c\u884c\u5584\u79ef\u5fb7\u662f\u6211\u672c\u8eab\u7684\u5174\u8da3\uff0c\u6240\u4ee5\u6276\u8001\u592a\u592a\u8fc7\u9a6c\u8def\u6211\u6bcf\u4e2a\u661f\u671f\u90fd\u505a\u4e00\u6b21\uff0c\u5982\u679c\u662f\u78b0\u5230\u56fd\u5b9a\u5047\u65e5\u7684\u8bdd\u6211\u8fd8\u505a\u591a\u4e24\u4e09\u6b21\u3002",
          "sentence_id": 10
        },
        {
          "_additional": {
            "distance": 934.29016
          },
          "sentence": "\u5c0f\u5f3a!\u5c0f\u5f3a\u4f60\u600e\u4e48\u4e86\u5c0f\u5f3a?\u5c0f\u5f3a\uff0c\u4f60\u4e0d\u80fd\u6b7b\u554a!\u6211\u8ddf\u4f60\u76f8\u4f9d\u4e3a\u547d\uff0c\u540c\u7518\u5171\u82e6\u4e86\u8fd9\u4e48\u591a\u5e74\uff0c\u4e00\u76f4\u628a\u4f60\u5f53\u4eb2\u751f\u9aa8\u8089\u4e00\u6837\u6559\u4f60\u517b\u4f60

In [36]:
# 输出结果
for i in response['data']['Get'][class_name]:
    print('='*20)
    print(i['sentence'])

除暴安良是我们做市民的责任，行善积德是我本身的兴趣，所以扶老太太过马路我每个星期都做一次，如果是碰到国定假日的话我还做多两三次。
小强!小强你怎么了小强?小强，你不能死啊!我跟你相依为命，同甘共苦了这么多年，一直把你当亲生骨肉一样教你养你，想不到今天，白发人送黑发人!
杀人这种事，我整天都有这种想法的。
本来我要用这10元钱念书，当个医生、律师，但为了世界和平……唉!
曾经有一份真诚的爱摆在我的面前，但是我没有珍惜，等到失去的时候才后悔莫及，尘世间最痛苦的事莫过 于此。如果上天可以给我个机会再来一次的话，我会对这个女孩说我爱她，如果非要在这份爱加上一个期限，我希望是一万年……
